### Checking output for correct grammer

In [1]:
from datasets import load_dataset
import json
import numpy as np
import os
import uptrain
from rouge import Rouge 
import random
from matplotlib import pyplot as plt
import pandas as pd
import subprocess
import zipfile

PyTorch is available but CUDA is not. Defaulting to SciPy for SVD


In [2]:
samsum_dataset = load_dataset("samsum")
dataset = samsum_dataset['test'][0:50]

  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
remote_url = "https://oodles-dev-training-data.s3.amazonaws.com/conversation_summarization_data.zip"
data_dir = 'data'
if not os.path.exists(data_dir):
    file_downloaded_ok = subprocess.call("wget " + remote_url, shell=True, 
                                         stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
    print("Data downloaded.")
    with zipfile.ZipFile('conversation_summarization_data.zip', 'r') as zip_ref:
        zip_ref.extractall("./")
    print("Prepared Model Outputs.")
    os.remove('conversation_summarization_data.zip')
else:
    print("Skipping data download as it already exists.")

Skipping data download as it already exists.


In [4]:
umap_check = {
    'type': uptrain.Visual.UMAP,
    "measurable_args": {
        'type': uptrain.MeasurableType.INPUT_FEATURE,
        'feature_name': 'bert_embs'
    },
    "label_args": [{
        'type': uptrain.MeasurableType.INPUT_FEATURE,
        'feature_name': 'dataset_label'
    },
    {
        'type': uptrain.MeasurableType.GRAMMAR_SCORE,
    }
    ],
    "hover_args": [
    {
        'type': uptrain.MeasurableType.INPUT_FEATURE,
        'feature_name': 'id'
    },
    {
        'type': uptrain.MeasurableType.PREDICTION,
    },
    {
        'type': uptrain.MeasurableType.INPUT_FEATURE,
        'feature_name': 'summary'
    },
    ],
    "update_freq": 50,
    # 'initial_dataset': reference_dataset_file,
    "do_clustering": False,
}

In [5]:
di_check = data_integrity_check = {
    "type": uptrain.Monitor.DATA_INTEGRITY,
    "measurable_args": {
        'type': uptrain.MeasurableType.GRAMMAR_SCORE,
    },
    "integrity_type": "grammar_check",
    "threshold": 60,
}

In [6]:
config = {
    "checks": [umap_check, di_check],
    "logging_args": {"st_logging": True},
    # ADD your OpenAI API key below
    "license_args": {"openai_key": "YOUR KEY HERE"}
}

framework = uptrain.Framework(cfg_dict=config)

Deleting contents of the folder:  uptrain_smart_data
Deleting contents of the folder:  uptrain_logs


In [ ]:
"""
Run the model in production and pass 
800 data points from SAMSum test.
"""

f = open(os.path.join(data_dir, f"out_test_samsum_summaries.json"))
all_summaries = json.load(f)
f.close()

"""
Note: We use sentence BERT embeddings generated from here:
https://huggingface.co/sentence-transformers
But any other embeddings, such as the ones generated by the
encoder can be used as well.
"""
f = open(os.path.join(data_dir, f"out_test_samsum_bert_embs.json"))
all_bert_embs = json.load(f)
f.close()

batch_size = 10
for idx in range(len(all_bert_embs)//batch_size):
    idxs = slice(idx*batch_size, (idx+1)*batch_size)
    this_batch = dataset['summary'][idxs]
    this_batch_dialog = dataset['dialogue'][idxs]

    inputs = {
        'id': list(range(idx*batch_size, (idx+1)*batch_size)),
        'bert_embs': np.array(all_bert_embs[idxs]),
        'dataset_label': ['samsum']*batch_size,
        'dialog': this_batch_dialog,
        'summary': this_batch,
    }
    idens = framework.log(inputs=inputs, outputs=all_summaries[idxs])
print(f"{(idx+1)*batch_size} predictions logged.")